In [2]:
import pandas as pd

# 1. Función universal anti-header
def limpia_anti_header(df):
    """
    Elimina cualquier fila donde el valor de una columna
    sea igual al nombre de esa columna (header infiltrado).
    También reemplaza vacíos, 'nan' y 'None' por NaN.
    """
    for col in df.columns:
        df = df[df[col] != col]
        df[col] = df[col].replace({'': pd.NA, 'nan': pd.NA, 'None': pd.NA})
    return df.reset_index(drop=True)

# 2. Rutas de los archivos
sell_in_file = 'sell-in.txt'
tb_stocks_file = 'tb_stocks.txt'
tb_productos_file = 'tb_productos.txt'
products_to_predict_file = 'product_id_apredecir201912.txt'

# 3. Leer los archivos (todo como string, para evitar problemas)
sell_in = pd.read_csv(sell_in_file, sep='\t', dtype=str)
tb_stocks = pd.read_csv(tb_stocks_file, sep='\t', dtype=str)
tb_productos = pd.read_csv(tb_productos_file, sep='\t', dtype=str)
products_to_predict = pd.read_csv(products_to_predict_file, sep='\t', header=None, dtype=str)
products_to_predict.columns = ['product_id']

# 4. Limpiar headers infiltrados en TODOS los DataFrames apenas los leés
sell_in = limpia_anti_header(sell_in)
tb_stocks = limpia_anti_header(tb_stocks)
tb_productos = limpia_anti_header(tb_productos)
products_to_predict = limpia_anti_header(products_to_predict)

# 5. LIMPIEZA extra de product_id (sacamos espacios, todo string)
for df in [sell_in, tb_stocks, tb_productos, products_to_predict]:
    df['product_id'] = df['product_id'].astype(str).str.strip()

# 6. Sacar los valores únicos de customer, producto y periodo
customer_ids = sorted(sell_in['customer_id'].unique())
product_ids = sorted(products_to_predict['product_id'].unique())
periodos = sorted(sell_in['periodo'].unique())

# 7. Crear el cubo completo customer x product x periodo SOLO con los productos a predecir
cubo = pd.MultiIndex.from_product(
    [customer_ids, product_ids, periodos],
    names=['customer_id', 'product_id', 'periodo']
).to_frame(index=False)

# 8. Merge con sell_in (LEFT JOIN: mantiene todos los registros del cubo)
cubo = cubo.merge(sell_in, on=['customer_id', 'product_id', 'periodo'], how='left', suffixes=('', '_sellin'))

# 9. Merge con tb_stocks según claves disponibles (customer_id, product_id, periodo)
claves_tb_stocks = [col for col in ['customer_id', 'product_id', 'periodo'] if col in tb_stocks.columns]
cubo = cubo.merge(tb_stocks, on=claves_tb_stocks, how='left', suffixes=('', '_stocks'))

# 10. Merge con tb_productos solo por product_id
cubo = cubo.merge(tb_productos, on='product_id', how='left', suffixes=('', '_productos'))

# 11. Limpiar headers infiltrados en el CUBO final (por si acaso)
cubo = limpia_anti_header(cubo)

# 12. Rellenar NaN en columnas numéricas con 0, strings con vacío
for col in cubo.select_dtypes(include='number').columns:
    cubo[col] = cubo[col].fillna(0)
for col in cubo.select_dtypes(include='object').columns:
    cubo[col] = cubo[col].fillna('')

# 13. Chequear resultado final
print("Columnas:", cubo.columns.tolist())
print("Primeras filas:\n", cubo.head())
print(f"Total de filas: {cubo.shape[0]}")

# 14. Guardar a CSV SIN headers infiltrados, ni index
cubo.to_csv('finalito.csv', index=False)


C:\Users\waldo\AppData\Local\Temp\ipykernel_28180\2339282230.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].replace({'': pd.NA, 'nan': pd.NA, 'None': pd.NA})


Columnas: ['customer_id', 'product_id', 'periodo', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'tn', 'stock_final', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size']
Primeras filas:
   customer_id product_id periodo plan_precios_cuidados cust_request_qty  \
0       10001      20001  201701                     0               11   
1       10001      20001  201702                     0               23   
2       10001      20001  201703                     0               33   
3       10001      20001  201704                     0                8   
4       10001      20001  201705                     0               15   

  cust_request_tn         tn stock_final cat1         cat2     cat3  brand  \
0        99.43861   99.43861               HC  ROPA LAVADO  Liquido  ARIEL   
1       198.84365  198.84365               HC  ROPA LAVADO  Liquido  ARIEL   
2        92.46537   92.46537               HC  ROPA LAVADO  Liquido  ARIEL   
3        13.29728   13.29728           

In [3]:
import pandas as pd

# Levantar el archivo CSV generado
datasetul = pd.read_csv('finalito.csv', dtype=str)

# Mostrar las primeras filas para revisar
print(datasetul.head())

# Ver cantidad de filas y columnas
print(f"Filas: {datasetul.shape[0]}, Columnas: {datasetul.shape[1]}")


  customer_id product_id periodo plan_precios_cuidados cust_request_qty  \
0       10001      20001  201701                     0               11   
1       10001      20001  201702                     0               23   
2       10001      20001  201703                     0               33   
3       10001      20001  201704                     0                8   
4       10001      20001  201705                     0               15   

  cust_request_tn         tn stock_final cat1         cat2     cat3  brand  \
0        99.43861   99.43861         NaN   HC  ROPA LAVADO  Liquido  ARIEL   
1       198.84365  198.84365         NaN   HC  ROPA LAVADO  Liquido  ARIEL   
2        92.46537   92.46537         NaN   HC  ROPA LAVADO  Liquido  ARIEL   
3        13.29728   13.29728         NaN   HC  ROPA LAVADO  Liquido  ARIEL   
4       101.20711  101.00563         NaN   HC  ROPA LAVADO  Liquido  ARIEL   

  sku_size  
0     3000  
1     3000  
2     3000  
3     3000  
4     3000  
Fi

In [4]:
datasetul

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,stock_final,cat1,cat2,cat3,brand,sku_size
0,10001,20001,201701,0,11,99.43861,99.43861,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000
1,10001,20001,201702,0,23,198.84365,198.84365,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000
2,10001,20001,201703,0,33,92.46537,92.46537,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000
3,10001,20001,201704,0,8,13.29728,13.29728,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000
4,10001,20001,201705,0,15,101.20711,101.00563,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16978675,10637,21276,201908,NaN,NaN,NaN,NaN,1.08488,PC,PIEL1,Cara,NIVEA,140
16978676,10637,21276,201909,NaN,NaN,NaN,NaN,0.87622,PC,PIEL1,Cara,NIVEA,140
16978677,10637,21276,201910,NaN,NaN,NaN,NaN,1.05889,PC,PIEL1,Cara,NIVEA,140
16978678,10637,21276,201911,NaN,NaN,NaN,NaN,1.06112,PC,PIEL1,Cara,NIVEA,140
